In [10]:
import zipfile
import os
from pathlib import Path



['15767 (Kawaki wo ameku - Aegis)', '15767 (Kawaki wo ameku - Aegis).zip', 'cryforrain.egg', 'ExpertStandard.dat', 'Info.dat', 'NormalStandard.dat', 'test.jpg']


In [11]:
#extract music data to specified folders, note, need to specify zippedsongs and songdata folders exist
allfiles = os.listdir(str(Path().absolute()) + "\zippedsongs")
print(allfiles)
rootdir = str(Path().absolute())
print(str(Path().absolute()) )
for mapname in allfiles:
    if mapname.endswith(".zip"):
        if not os.path.exists(rootdir+"/songdata/" + mapname[:-4]):
            os.mkdir(rootdir+"/songdata/" + mapname[:-4])
        zip_ref = zipfile.ZipFile(rootdir + "\\zippedsongs\\"+ mapname) # create zipfile object
        zip_ref.extractall(rootdir + "\\songdata\\" + mapname[:-4]) # extract file to dir
        zip_ref.close() # close file

        

['15767 (Kawaki wo ameku - Aegis).zip', '159be (coffee and boba - moonunit7).zip']
c:\Users\vedan\Documents\GitHub\Beat_Saber_Map_Generator_APS360_Project\Note_Orientation


In [12]:
import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])
install("pandas")
import pandas as pd 

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\Users\vedan\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [ ]:
import json
import numpy as np

In [ ]:

def loadmaps():
    output = list(list())
    rootdir = str(Path().absolute())
    for subdir, dirs, files in os.walk(os.path.join(rootdir,"songdata")):
        for file in files:
            if file.startswith("Expert"):
                with open(os.path.join(rootdir,"songdata",subdir,file), 'rb') as f:
                    loadedmapdata = json.load(f)
                    output.append([subdir.split("\\")[-1], loadedmapdata])
    print(output[0][0])
    return output

In [28]:

def create_processed_data(map_files):
    wide_beat_maps = list()
    for beatmap in (map_files):
        notemap = pd.DataFrame.from_dict(beatmap[1]['_notes']).add_prefix('notes')
        notelist = list()
        widenote = None
        for i in range(len(notemap['notes_type'].unique())):
            if i == 0:

                #Make separate dataframe for first note type and add a suffix for the column names
                note_type_df = notemap[notemap['notes_type'] == notemap['notes_type'].unique()[i]].reset_index()
                note_type_df.drop('index', axis = 1, inplace=True)
                note_type_df = note_type_df.add_suffix(f"_{notemap['notes_type'].unique()[i]}")
                note_type_df['_time'] = note_type_df[f"notes_time_{notemap['notes_type'].unique()[i]}"]
                note_type_df.drop(f"notes_time_{notemap['notes_type'].unique()[i]}", axis = 1, inplace = True)  
                    
                notes_b = notemap[notemap['notes_type'] == notemap['notes_type'].unique()[i+1]].reset_index()
                notes_b.drop('index', axis = 1, inplace=True)
                notes_b = notes_b.add_suffix(f"_{notemap['notes_type'].unique()[i+1]}")
                notes_b['_time'] = notes_b[f"notes_time_{notemap['notes_type'].unique()[i+1]}"]
                notes_b.drop(f"notes_time_{notemap['notes_type'].unique()[i+1]}", axis = 1, inplace = True)
                widenote = pd.merge(note_type_df, notes_b, on = '_time', how = 'outer', sort = True)
            elif i ==1:
                continue
            elif i>1:
                notes_c = notemap[notemap['notes_type'] == notemap['notes_type'].unique()[i]].reset_index()
                notes_c.drop('index', axis = 1, inplace=True)
                notes_c = notes_c.add_suffix(f"_{notemap['notes_type'].unique()[i]}")
                notes_c['_time'] = notes_c[f"notes_time_{notemap['notes_type'].unique()[i]}"]
                notes_c.drop(f"notes_time_{notemap['notes_type'].unique()[i]}", axis = 1, inplace = True)
                widenote = pd.merge(widenote, notes_c, on = '_time', how = 'outer', sort = True)
        #Replace NaN with 999
        widenote.fillna(999, inplace = True)
        #Coerce all columns except _time back to integer
        for col in widenote.columns:
            if col != '_time':
                widenote[col] = widenote[col].astype(int)
        
        song_len = widenote['_time'].max()
        expand_time = pd.DataFrame(np.arange(0.00, song_len, 0.0625), columns = ['_time'])
        expanded_notes = pd.merge(widenote, expand_time, on='_time', how = 'outer', sort = True)
        #Replace NaN with 999
        expanded_notes.fillna(999, inplace = True)
        #Coerce all columns except _time back to integer
        for col in expanded_notes.columns:
            if col != '_time':
                expanded_notes[col] = expanded_notes[col].astype(int)
        wide_beat_maps.append([beatmap[0],expanded_notes])
    return wide_beat_maps

processed_dataframe_list = create_processed_data(loadmaps())
print(processed_dataframe_list)



15767 (Kawaki wo ameku - Aegis)
['15767 (Kawaki wo ameku - Aegis)',       notes_lineIndex_1  notes_lineLayer_1  notes_type_1  \
0                   999                999           999   
1                   999                999           999   
2                   999                999           999   
3                   999                999           999   
4                   999                999           999   
...                 ...                ...           ...   
9476                999                999           999   
9477                  2                  1             1   
9478                  2                  1             1   
9479                  3                  0             1   
9480                  3                  0             1   

      notes_cutDirection_1       _time  notes_lineIndex_0  notes_lineLayer_0  \
0                      999    0.000000                999                999   
1                      999    0.062500             